In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from sklearn import metrics
from tensorflow.keras import backend as K


2024-06-03 12:45:46.977495: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-03 12:45:46.977609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-03 12:45:49.275115: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-03 12:45:53.144315: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 12:46:03.148062: W tensorflow/comp

In [9]:
img_rows, img_cols = 28, 28
num_classes = 10
batch_size = 128
epochs = 1

(x_train, y_train), (x_test, y_test) = mnist.load_data()


11490434/11490434 [==============================] - 0s 0us/step


In [10]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


In [11]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


2024-06-03 12:46:41.805127: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


469/469 [==============================] - 104s 216ms/step - loss: 0.2415 - accuracy: 0.9273 - val_loss: 0.0515 - val_accuracy: 0.9827


In [12]:
import os
!mkdir "keras_model"
save_path = "./keras_model/"

model.save_weights(os.path.join(save_path,"model-weights.h5"))

model_json = model.to_json()
with open(os.path.join(save_path,"model.json"), "w") as json_file:
    json_file.write(model_json)

In [33]:
!tar -czvf model.tar.gz keras_model

keras_model/
keras_model/model-weights.h5
keras_model/model.json
keras_model/.ipynb_checkpoints/


In [34]:
import boto3
import sagemaker
sess = sagemaker.Session()
uri = sess.upload_data(path='model.tar.gz')
uri

's3://sagemaker-us-east-1-179201351861/data/model.tar.gz'

In [ ]:
from sagemaker.tensorflow import TensorFlowModel

model = TensorFlowModel(model_data=uri, framework_version="2.2.0", role=get_execution_role())

predictor = model.deploy(initial_instance_count=1, instance_type='ml.c5.xlarge')


--------------------------------------